In [10]:
import os

In [11]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

In [12]:
!nividia-sim

/bin/bash: line 1: nividia-sim: command not found


In [13]:
# torch.cuda.empty_cache()

# install_librarys

In [14]:
import wandb
import re
import torch
from datasets import DatasetDict,Dataset,load_dataset
from transformers import (AutoTokenizer ,
                          AutoModelForMaskedLM,
                          DataCollatorForLanguageModeling,
                          TrainingArguments,
                          Trainer)
import spacy
from spacy.tokens import Doc

In [15]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: a48104046 (a48104046-dfas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# datasets

In [16]:
datasets = load_dataset("HuggingFaceFW/fineweb-2","arz_Arab",streaming=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [17]:
for split_name in datasets.keys():
  datasets[split_name] = datasets[split_name].take(18000)


In [18]:
datasets

IterableDatasetDict({
    test: IterableDataset({
        features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'language_script', 'minhash_cluster_size', 'top_langs'],
        num_shards: 1
    })
    train: IterableDataset({
        features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'language_script', 'minhash_cluster_size', 'top_langs'],
        num_shards: 1
    })
})

In [19]:
dataset = DatasetDict({"train":Dataset.from_generator(lambda:datasets["train"]),
             "test":Dataset.from_generator(lambda:datasets["test"])})

# configration

In [89]:
class Config:
  def __init__(self):
    self.model_name = "answerdotai/ModernBERT-base"
    self.max_length =512
    self.new_tokenizer_vocab_size = 64000
    self.mlm_probability =0.15
## Training Arguments
    self.base_dir = "/content/ModernEgyBert"
    self.output_dir = self.base_dir+"/egyptian_modernbert_fineweb2"

    self.num_train_epoch = 3
    self.per_device_train_batch_size = 4
    self.per_device_eval_batch_size = 2
    self.evaluation_strategy = "steps"
    self.eval_steps = 3000
    self.logging_steps = 3000
    self.save_steps = 3000
    self.save_total_limit = 2
    self.learnin_rate = 5e-5
    self.warmup_steps = 500
    self.weight_decay = 0.01
    self.report_to = "wandb"
    self.run_anme = "modernbert-egyptian-handle-english"
    self.overwrite_output_dir = True
    self.hidden_size = 768 # Added hidden size
    self.pad_token_id = 0 # Added pad_token_id



In [100]:
config = Config()

In [22]:
config.num_train_epoch

3

In [23]:
config.run_anme

'modernbert-egyptian-handle-english'

# Let's go

In [24]:
datasets

IterableDatasetDict({
    test: IterableDataset({
        features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'language_script', 'minhash_cluster_size', 'top_langs'],
        num_shards: 1
    })
    train: IterableDataset({
        features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'language_script', 'minhash_cluster_size', 'top_langs'],
        num_shards: 1
    })
})

In [25]:
print(f"language is {next(iter(datasets['train']))['language']}, text is {next(iter(datasets['train']))['text']}")

language is arz, text is November 2011
You have no idea how hard I’ve looked for a gift to bring You. Nothing...– Rumi
أسرة
عندى كام فكرة كده على فكرة الأسرة جميلة فكرة العيلة و ان يكون عندك أهل بيحبوك و تحبهم .. و يخافوا عليك و كده نعمة من نعم الله علينا و حقيقى ان فى “الغرب” (و أنا بكره الكلمة دى) افتقدوا معانى كتير مرتبطة بالتنظيم ده أنا مثلا بحب أوى لمة عيلتنا الكبيرة كل يوم خميس أو جمعة بحب لما أكل أنا و ماما و بابا على تربيزة المطبخ مع بعض بحب احكى مع ماما على اللى بفكر و بحلم به .. بس مؤخرا...
Music is a moral law. It gives soul to the universe, wings to the mind, flight...– Plato (via quote-book)
لما تيجي مريم
لما تيجي مريم حديها حاجات كتير أوي حديها العلبة الصدف المتبطنة بالقطيفة… اللي جاتلي هدية من سوريا.. لما كنت بتمشى أنا و هبة و مها في ساحة الأمويين ساعتها اشترينا غوايش زرقا زي بعض و لما آلاء بعد كده جت.. زعلت اننا مجيبنلهاش غويشة زرقا زينا فلما عدينا على محل تاني… جبت 3 غوايش ملونين تانيين ليا أنا و وئام و آلاء خمس صبايا… من بلودان و قدسيا و حلب و دير الزور و القاهرة ...


In [26]:
# DatasetDict({"train":Dataset.from_generator(lambda:datasets["train"]),
#              "test":Dataset.from_generator(lambda:datasets["test"])})

In [27]:
nlp = spacy.load("en_core_web_sm")

In [28]:
def remove_english_words(text):
    text_no_eng = re.sub(r"\b[a-zA-Z]+\b", "", text)
    return text_no_eng.strip()

def preprocess_examples(examples):
    new_texts = []
    for t in examples["text"]:
        new_texts.append(remove_english_words(t))
    # Return a dictionary with the same column name
    return {"text": new_texts}

# Apply this preprocessing to train & test
datasets = dataset.map(preprocess_examples, batched=True)

In [29]:
# remove_english_words(next(iter(datasets["train"]))["text"])

In [30]:
# Apply this preprocessing to train & test
dataset = datasets.map(preprocess_examples, batched=True)

# Using Spacy

In [31]:
# # Load spaCy's English model
# nlp = spacy.load("en_core_web_sm")

# def remove_english_words(text):
#     """Removes English words using spaCy's language detection."""
#     doc = nlp(text)
#     return " ".join([token.text for token in doc if token.lang_ != "en"]).strip()

# def preprocess_examples(examples):
#     """Applies English word removal to each text example."""
#     new_texts = [remove_english_words(t) for t in examples["text"]]
#     return {"text": new_texts}

In [32]:
# text = """
# You have no idea how hard I’ve looked for a gift to bring You. Nothing...– Rumi
# أسرة
# عندى كام فكرة كده على فكرة الأسرة جميلة فكرة العيلة و ان يكون عندك أهل بيحبوك و تحبهم .. و يخافوا عليك و كده نعمة من نعم الله علينا و حقيقى ان فى “الغرب” (و أنا بكره الكلمة دى) افتقدوا معانى كتير مرتبطة بالتنظيم ده أنا مثلا بحب أوى لمة عيلتنا الكبيرة كل يوم خميس أو جمعة بحب لما أكل أنا و ماما و بابا على تربيزة المطبخ مع بعض بحب احكى مع ماما على اللى بفكر و بحلم به .. بس مؤخرا...
# Music is a moral law. It gives soul to the universe, wings to the mind, flight...– Plato (via quote-book)
# لما تيجي مريم
# لما تيجي مريم حديها حاجات كتير أوي حديها العلبة الصدف المتبطنة بالقطيفة… اللي جاتلي هدية من سوريا.. لما كنت بتمشى أنا و هبة و مها في ساحة الأمويين ساعتها اشترينا غوايش زرقا زي بعض و لما آلاء بعد كده جت.. زعلت اننا مجيبنلهاش غويشة زرقا زينا فلما عدينا على محل تاني… جبت 3 غوايش ملونين تانيين ليا أنا و وئام و آلاء خمس صبايا… من بلودان و قدسيا و حلب و دير الزور و القاهرة ...
# أحب الناس الذين يعتبرون أن وجودهم قصة ولا يتماهون مع هذا الوجود بجدية مفرطة– سحر مندور “سأرسم نجمة على جبين فيينا” (via amrezzat)
# Writing is a form of therapy; sometimes I wonder how all those who do not write,...– Graham Greene
# لما الشتا يدق البيبان
# هوا ساقع بيلمس وشك و دفا فى القلب لابس حلو .. اصل لبس الشتا كله بيبقى شكله حلو .. بيبقى شكلك شيك تمشى فى شوارع القاهرة .. و هى مش زاحمة اوى و لا فاضية اوى النور و انعكاسه على وشوش الناس شباب بيعدوا فى نص الشارع بيكلوا سندويتشات زحمة على الأرصفة و ناس قاعدين يشربوا حاجة سخنة تدفيهم على كوبرى قصر النيل ولد و بنت .. ايديهم فى ايدين بعض مالكين العالم باللى فيه يمكن ساكتين .. يمكن بيتكلموا ...
# حنين لأسكندرية– .
# ناقص حاجة !!
# :)
# حاسة بسكينة و هدوء من فترة محستوش جبت حلقان حلوة لبنات خلاتى و شفت أسكندريلا بيغنوا لأول مرة غنيت بصوت عالى و بكل القوة اللى عندى معاهم .. شيد قصورك و اتجمعوا العشاق و يحكى ان كنت بغنى كأنى فى مظاهرة أنا أصلا بقالى حبة عارفة إنى محتاجة مظاهرة اهتف و اطلع فيها طاقة الغضب اللى جوايا الشوارع كانت فاضية :)) شفت أصحابى و كنت حاطة روج فوشيا :)))))...
# :)
# :)

# """


# Tokenizer

In [33]:
base_tokenizer = AutoTokenizer.from_pretrained(config.model_name ,use_fast=True)

In [34]:
# We'll create a batch iterator to feed lines to the tokenizer trainer
def batch_iterator(ds,batch_size=1000):
  for t in range(0,len(ds),batch_size):

    yield ds[t:t+batch_size]["text"]

In [35]:
#  Train a new tokenizer from the cleaned training text
train_iterator = batch_iterator(dataset["train"])
new_tokenizer = base_tokenizer.train_new_from_iterator(train_iterator,
                                                       vocab_size= config.new_tokenizer_vocab_size,
                                                       show_progress=True)
new_tokenizer.save_pretrained(f"{config.base_dir}/tokenizer")

('/content/ModernEgyBert/tokenizer/tokenizer_config.json',
 '/content/ModernEgyBert/tokenizer/special_tokens_map.json',
 '/content/ModernEgyBert/tokenizer/tokenizer.json')

In [36]:
new_tokenizer = new_tokenizer.from_pretrained(f"{config.base_dir}/tokenizer",use_fast=True)

In [37]:
def tokenize_function(examplses):
  return new_tokenizer(
      examplses["text"],padding="max_length",
      truncation=True,
      max_length=config.max_length)

In [38]:
column_names = dataset["train"].column_names
train_dataset = dataset["train"].map(tokenize_function,batched=True,remove_columns=column_names)
test  = dataset["test"].map(tokenize_function,batched=True,remove_columns=column_names)

Map:   0%|          | 0/9216 [00:00<?, ? examples/s]

In [39]:
print(train_dataset)
print(test)

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 18000
})
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 9216
})


In [106]:
data_collator = DataCollatorForLanguageModeling(tokenizer = new_tokenizer,
                                                mlm=True,
                                                mlm_probability=config.mlm_probability)
model = AutoModelForMaskedLM.from_pretrained(config.model_name)

In [108]:
model.resize_token_embeddings(len(new_tokenizer))

Embedding(20000, 768)

In [109]:
embedding_layer =model.get_input_embeddings()

In [110]:
embedding_layer #(vocabsize,emdings_dim)

Embedding(20000, 768)

In [111]:
vocabsize,emdings_dim = embedding_layer.weight.shape

In [112]:
vocabsize

20000

In [113]:
emdings_dim

768

In [47]:
# all number of parameters
embedding_layer.weight.numel()

15360000

In [117]:
training_args = TrainingArguments(
    output_dir = config.output_dir,
    overwrite_output_dir = config.overwrite_output_dir,
    num_train_epochs = 1,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 3,
    evaluation_strategy = config.evaluation_strategy,
    eval_steps = config.eval_steps,
    logging_steps = config.logging_steps,
    save_steps = config.save_steps,
    save_total_limit = config.save_total_limit,
    learning_rate = config.learnin_rate,
    warmup_steps = config.warmup_steps,
    weight_decay = config.weight_decay,
    report_to = config.report_to,
    run_name = config.run_anme,
    fp16 = True,
    gradient_accumulation_steps = 2,


)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1609: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [118]:
trainer = Trainer(
    args= training_args,
    model = model,
    train_dataset = train_dataset,
    eval_dataset = test,
    data_collator = data_collator,
)

In [119]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [62]:
torch.cuda.empty_cache()

In [120]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=2250, training_loss=12.841714409722222, metrics={'train_runtime': 1047.4677, 'train_samples_per_second': 17.184, 'train_steps_per_second': 2.148, 'total_flos': 6134662103040000.0, 'train_loss': 12.841714409722222, 'epoch': 1.0})

In [121]:
from huggingface_hub import login



In [75]:
login()

In [122]:
# #Push to Hugging Face Hub:
trainer.push_to_hub("Modern-EgyBert")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/505M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ah7med/egyptian_modernbert_fineweb2/commit/755538140fa0c4b0285d6a6168e8f0f5c487d7e1', commit_message='Modern-EgyBert', commit_description='', oid='755538140fa0c4b0285d6a6168e8f0f5c487d7e1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ah7med/egyptian_modernbert_fineweb2', endpoint='https://huggingface.co', repo_type='model', repo_id='Ah7med/egyptian_modernbert_fineweb2'), pr_revision=None, pr_num=None)

In [ ]:
#COMMIT]
trainer.commit_to_hub("Modern-EgyBert_add")

In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("Ah7med/egyptian_modernbert_fineweb2")
model = AutoModelForMaskedLM.from_pretrained("Ah7med/egyptian_modernbert_fineweb2")

# Then create the pipeline with these components
pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer)

OSError: Can't load tokenizer for 'Ah7med/egyptian_modernbert_fineweb2'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'Ah7med/egyptian_modernbert_fineweb2' is the correct path to a directory containing all relevant files for a PreTrainedTokenizerFast tokenizer.

In [ ]:
demo = "صباح الفل يا [MASK]"
result = pipe(demo)

In [ ]:
result

In [ ]:
for r in result:
  print(f"token {r["sequence"]}",f"score {r["score"]}")

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("Ahmed/egyptian_modernbert_fineweb2")
model = AutoModelForMaskedLM.from_pretrained("Ahmed/egyptian_modernbert_fineweb2")